In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
#from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from sklearn.metrics import accuracy_score
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications.resnet import ResNet152
#from tensorflow.
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet import preprocess_input


In [2]:
# Load the CIFAR-100 dataset
from tensorflow.keras.datasets import cifar100
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

169001437/169001437 [==============================] - 13s 0us/step


In [3]:
# Preprocess the input data
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

In [4]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [5]:
base_model = ResNet152(include_top=False, weights='imagenet', input_shape=(32, 32, 3))

234698864/234698864 [==============================] - 11s 0us/step


In [6]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

In [7]:
# Add a global average pooling layer and a dense output layer
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(100, activation='softmax')(x)

In [8]:
# Define the model
model = tf.keras.models.Model(inputs=base_model.input, outputs=x)

In [10]:
#ine-tune the top layers
for layer in model.layers[-3:]:
    layer.trainable = True

In [11]:
from keras.backend import dropout
learning_rates = [0.001, 0.0001, 0.1]
batch_sizes = [32, 64, 128]
num_epochs = 7
#dropout= [0.3]
for lr in learning_rates:
    for batch_size in batch_sizes:
          model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                          loss='sparse_categorical_crossentropy',
                          metrics=['accuracy'])
          model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(x_test, y_test))
          # Evaluate the model on the test data
          y_pred = model.predict(x_test)
          y_pred_classes = np.argmax(y_pred, axis=1)
          accuracy = accuracy_score(y_test, y_pred_classes)
          print(f'Learning rate: {lr}, Batch size: {batch_size} Accuracy: {accuracy}')




Epoch 1/7
1563/1563 [==============================] - 63s 34ms/step - loss: 3.9684 - accuracy: 0.1009 - val_loss: 3.9896 - val_accuracy: 0.1006
Epoch 2/7
1563/1563 [==============================] - 51s 32ms/step - loss: 3.9440 - accuracy: 0.1040 - val_loss: 3.9796 - val_accuracy: 0.0975
Epoch 3/7
1563/1563 [==============================] - 48s 31ms/step - loss: 3.9226 - accuracy: 0.1080 - val_loss: 3.9697 - val_accuracy: 0.1030
Epoch 4/7
1563/1563 [==============================] - 49s 32ms/step - loss: 3.9074 - accuracy: 0.1109 - val_loss: 3.9509 - val_accuracy: 0.1065
Epoch 5/7
1563/1563 [==============================] - 51s 33ms/step - loss: 3.8923 - accuracy: 0.1139 - val_loss: 3.9459 - val_accuracy: 0.1088
Epoch 6/7
1563/1563 [==============================] - 50s 32ms/step - loss: 3.8711 - accuracy: 0.1157 - val_loss: 3.9775 - val_accuracy: 0.1025
Epoch 7/7
313/313 [==============================] - 11s 24ms/step
Learning rate: 0.001, Batch size: 32 Accuracy: 0.105
Epoch 1/7
